# Extracting Final 10-K links and Keywords


This Chunk of Code Imports Libraries

In [ ]:
import pandas as pd
import urllib2
from bs4 import SoupStrainer
from bs4 import BeautifulSoup
import re

df = pd.read_csv('health sentiment.csv') #Importing the companies list by industry to get the final link for 10K filing

Extracting HTML links for compaines

In [ ]:
df['html'] = 'TBD' #Initializing empty colun html
df.head()

In [ ]:
import time
import re
import pandas as pd
from urllib import FancyURLopener  # This is library that helps us create the headless browser
from random import choice #This library helps pick a random item from a list

user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

class MyOpener(FancyURLopener, object):
    version = choice(user_agents)

myopener = MyOpener()

page=myopener.open('https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&company=FOREST+CITY+ENTERPRI&type=10-K&dateb=&owner=exclude&count=100')
html = page.read()

This Entire chunks appends html links that will be scrapes for keywords in subsequent sections

In [ ]:
for j in range(0,len(df)):
    myopener = MyOpener()
    #print(df['weblink'][j])
    page=myopener.open(df['weblink'][j])
    html = page.read()
    soup = BeautifulSoup(html, 'lxml')  
    x = soup.find_all('td')
    for i in range(0,len(x)):
        #print x[i].get_text()
        if(x[i].get_text() == '10-K' and x[i+2].get_text() == '10-K'):
           # print j,i
            df['html'][j] = 'https://www.sec.gov' + x[i+1].a['href']
            break
        if(x[i].get_text() == '10-K' and x[i-2].get_text() != '10-K'):
           # print j,i
            df['html'][j] = 'https://www.sec.gov' + x[i-1].a['href']
            break

In [ ]:
df.to_csv('html_links_fin.csv', sep = ',', header = True, index = False) #Writes files with links

Subsequent chunk of code gets the keywords and their frequencies

In [ ]:
#Inititalizing Keywords
keywords = ['analytics', 'big data', 'big-data', 'bitcoin', 'blockchain', 'emerging technologies', 'artificial intelligence', 'neural network', 'cloud', 'data driven', 'storage system', 'cutting edge', 'cutting-edge', 'digital transformation', 'digital innovation', 'digital marketing', 'technological improvement', 'latest technology', 'disruption', 'startup', 'crowdfunding', 'financial technology', 'robotic', 'digital currency', 'natural language', 'cell technology', 'vaccine technology', 'health technology', 'polymer technology', 'drug delivery technology']

In [ ]:
#Inititalizing the columns for keywords
for col in keywords:
    df[col] = 'TBD'

The subsequent chunk finds the keywords in each 10-K and appends it in respective column

In [ ]:
for temp in range(0,len(df)):
    print temp
    myopener = MyOpener()
    #print(df['weblink'][j])
    page=myopener.open(df['html'][temp])
    html = page.read()
    for cols in keywords:
        df[cols][temp] = len([m.start() for m in re.finditer( cols, html.lower())])
    

In [ ]:
df.to_csv('health sentiment.csv', sep = ',', header = True , index = False) #Writes the file to the industry specific file